In [1]:
import pandas as pd
import urllib.request
%matplotlib inline
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import Counter

print("準備完了")


準備完了


# データを取るためにclass生成

In [2]:
class ThubanNaver:

    X_train, y_train, X_test, y_test = [], [], [], []
    index_to_word, word_to_index = {}, {}
    

    def __init__(self):
        print("생성")
        
    def set_init_words(self):
        # set word dictionary
        index_to_word = pd.read_csv('words.csv')
        self.index_to_word = {idx:word for idx, word in index_to_word['word'].items()}
        self.word_to_index = {word:idx for idx, word in self.index_to_word.items()}
        
    def set_init_datas(self):
        read_train = pd.read_csv('train.csv').dropna()
        read_test = pd.read_csv('test.csv').dropna()
        
        self.X_train = [self.get_encoded_sentence(text) for idx, text in read_train['text'].items()]
        self.X_test = [self.get_encoded_sentence(text) for idx, text in read_test['text'].items()]
        
        self.y_train = np.array([label for idx, label in read_train['label'].items()])
        self.y_test = np.array([label for idx, label in read_test['label'].items()])
        
    def save_datas_to_csv(self):
        
        # 데이터를 읽어봅시다. 
        train_txt = pd.read_table('ratings_train.txt')
        test_txt = pd.read_table('ratings_test.txt')
        # 重複のデータを削除, そしてNANを削除
        train_txt = train_txt.drop_duplicates('document').dropna()
        test_txt = test_txt.drop_duplicates('document').dropna()
        
        
        okt = Okt()
        
        stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
        
        #セーブされる配列
        train_texts = []
        test_texts = []
        clean_words = []
        

        # 単語辞書をcsvファイルにセーブために韓国語だけまたはtoken化する
        for i in range(0,2):
            if i == 0:
                read_x_datas = train_txt['document']
                save_x_datas = train_texts
            else:
                read_x_datas = test_txt['document']
                save_x_datas = test_texts
                
            for idx, item in enumerate(read_x_datas.items()):
                item_text = re.compile('[가-힣]+').findall(item[1])
                results = okt.morphs(' '.join(item_text))
                stopwords_result = []
                for result in results:
                    if result not in stopwords: #単語たちのstopwordsを削除
                        stopwords_result.append(result)
                
                # stopwordsしても文字がある場合だけ
                if(len(stopwords_result) > 0):
                    save_x_datas.append(' '.join(stopwords_result)) # trainまたはtestデータ
                    clean_words.extend(stopwords_result) #辞書のcsvのため       
                # print("{} {}".format(results, idx))
                
                # 全部forしたら時間がかかるので（テスト用）
                #if idx == 99:
                #    break

        #単語たちの重複を削除
        clean_words = set(clean_words)
        clean_words = list(clean_words)
        
        #
        clean_words.insert(0, "<UNUSED>")
        clean_words.insert(0, "<UNK>")
        clean_words.insert(0, "<BOS>")
        clean_words.insert(0, "<PAD>")
        
        
        # words.csvセーブ
        pd.DataFrame(clean_words, columns=['word']).to_csv('words.csv', encoding="utf-8")
        self.set_init_words() #下のtexts, labelsをセーブするため (classの変数 index_to_word, word_to_indexに 実際の値を入力)
        
        
        # trainとtestをcsvでセーブする
        train_labels = [result for result in train_txt['label']][:len(train_texts)]
        test_labels = [result for result in test_txt['label']][:len(test_texts)]

        train_df = pd.DataFrame({'text':train_texts, 'label':train_labels})
        test_df = pd.DataFrame({'text':test_texts, 'label':test_labels})
        
        pd.DataFrame(train_df).to_csv('train.csv', encoding="utf-8")
        pd.DataFrame(test_df).to_csv('test.csv', encoding="utf-8")
        self.set_init_datas() # classの変数 X_train, y_train, X_test, y_testに 実際の値を入力
        
        
    def load_csv_datas(self):
        self.set_init_words()
        self.set_init_datas()
        
    def load_data(self, num_words):
        if len(self.word_to_index.items()) < num_words: #num_words数がword_to_index配列の長さより長い場合はword_to_indexの長さに従う
            num_words = len(self.word_to_index.items())
        return self.X_train, self.y_train, self.X_test, self.y_test, [ item for item, idx in self.word_to_index.items() if idx < int(num_words) ]
        
        
    # 문장 1개를 활용할 딕셔너리와 함께 주면, 단어 인덱스 리스트 벡터로 변환해 주는 함수입니다. 
    # 단, 모든 문장은 <BOS>로 시작하는 것으로 합니다. 
    def get_encoded_sentence(self, sentence):
        return [self.word_to_index['<BOS>']]+[self.word_to_index[word] if word in self.word_to_index else self.word_to_index['<UNK>'] for word in sentence.split()]

    # 여러 개의 문장 리스트를 한꺼번에 단어 인덱스 리스트 벡터로 encode해 주는 함수입니다. 
    def get_encoded_sentences(self, sentences):
        return [self.get_encoded_sentence(sentence) for sentence in sentences]
    
    # 숫자 벡터로 encode된 문장을 원래대로 decode하는 함수입니다. 
    def get_decoded_sentence(self, encoded_sentence):
        return ' '.join(self.index_to_word[index] if index in self.index_to_word else '<UNK>' for index in encoded_sentence[1:])  #[1:]를 통해 <BOS>를 제외

    # 여러개의 숫자 벡터로 encode된 문장을 한꺼번에 원래대로 decode하는 함수입니다. 
    def get_decoded_sentences(self, encoded_sentences):
        return [self.get_decoded_sentence(encoded_sentence, self.index_to_word) for encoded_sentence in encoded_sentences]

TN = ThubanNaver()

생성


'ratings_train.txt'と'ratings_test.txt'だけで   
X_train, y_train, X_test, y_test, words_to_indexの値を得るためのclass

# データ加工

In [ ]:
print("スタート")
TN.save_datas_to_csv()
print("データセーブ、ロード完了")

In [3]:
# すでにTN.save_datas_to_csv()をして words.csv, x_test.csv, x_train.csv ファイルが居る場合は下の関数を利用してデータを読み込みする
TN.set_init_words()
TN.set_init_datas()
print("データロード完了")


データロード完了


データを初めて扱っているならTN.save_datas_to_csv()関数を   
そうじゃなければなければ   
TN.set_init_words()   
TN.set_init_datas()   
を使ってください   
   
### こうやって二つのコードに割った理由は TN.save_datas_to_csv() この関数が結構時間がかかるため

# データ獲得

In [4]:
X_train, y_train, X_test, y_test, words_to_index = TN.load_data(num_words=113661)
print("훈련 샘플 개수: {}, 테스트 개수: {}".format(len(X_train), len(X_test)))
print("単語数: {}".format(len(words_to_index)))

훈련 샘플 개수: 145237, 테스트 개수: 48773
単語数: 113661


load_data(num_words=n)のnum_wordsで単語を何個持ってくるのかを決めます   
存在する単語数より多い場合は存在する単語すべてを持ってきます

# モデル作成

In [6]:
vocab_size = 113661    # 어휘 사전의 크기입니다
word_vector_dim = 16 # 워드 벡터의 차원수 (변경가능한 하이퍼파라미터)

# 모델 설계. 위에서 만든 모델을 사용해봅시다.
model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None, )))
model.add(keras.layers.Conv1D(16, 7, activation='relu'))
model.add(keras.layers.MaxPooling1D(5))
model.add(keras.layers.Conv1D(16, 7, activation='relu'))
model.add(keras.layers.GlobalMaxPooling1D())
model.add(keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid')) # 최종 출력은 긍정/부정을 나타내는 1dim
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          1818576   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 16)          1808      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 16)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 16)          1808      
_________________________________________________________________
global_max_pooling1d (Global (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 8)                 136       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                

# 配列の長さをお互い同じくする

In [7]:
total_data_text = list(X_train) + list(X_test)
# 텍스트데이터 문장길이의 리스트를 생성한 후
num_tokens = [len(tokens) for tokens in total_data_text]
num_tokens = np.array(num_tokens)
# 문장길이의 평균값, 최대값, 표준편차를 계산해 본다. 
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))

# 예를들어, 최대 길이를 (평균 + 2*표준편차)로 한다면,  
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
maxlen = int(max_tokens)
print('pad_sequences maxlen : ', maxlen)
print('전체 문장의 {}%가 maxlen 설정값 이내에 포함됩니다. '.format(np.sum(num_tokens < max_tokens) / len(num_tokens)))

문장길이 평균 :  12.058857790835525
문장길이 최대 :  73
문장길이 표준편차 :  9.270684715127398
pad_sequences maxlen :  30
전체 문장의 0.9344982217411474%가 maxlen 설정값 이내에 포함됩니다. 


# 配列の長さをお互い同じくする

In [8]:

X_train = keras.preprocessing.sequence.pad_sequences(X_train,
                                                        value=TN.word_to_index["<PAD>"],
                                                        padding='pre', # 혹은 'pre'
                                                        maxlen=maxlen)

X_test = keras.preprocessing.sequence.pad_sequences(X_test,
                                                       value=TN.word_to_index["<PAD>"],
                                                       padding='pre', # 혹은 'pre'
                                                       maxlen=maxlen)

embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=word_vector_dim, mask_zero=True)


X_train = embedding(X_train)
X_test = embedding(X_test)

print("clear")

clear


配列の長さをお互い同じくしなければ   
モデルの訓練ができなくなります   
   
そしてpad_sequencesの関数の中でpadding='pre'はpreがいいらしい   
X_train、X_testの配列一つ一つには文章があるんですが個々長さが違うので後ろのには空白が出ます   
モデルの訓練の時空白が最後に計算して悪い影響が出るそうです   
なので列の前の方に空白を置いて後ろは実際の文字を置くpreがいいらしいです

In [9]:
X_train[0]

<tf.Tensor: shape=(30, 16), dtype=float32, numpy=
array([[ 0.04613856,  0.01403167,  0.02923531, -0.02850889,  0.03368899,
         0.0168766 ,  0.01934865, -0.04111039,  0.01275269,  0.04354442,
         0.03784927, -0.00735014, -0.04301777,  0.04044653,  0.03388585,
         0.02398138],
       [ 0.04613856,  0.01403167,  0.02923531, -0.02850889,  0.03368899,
         0.0168766 ,  0.01934865, -0.04111039,  0.01275269,  0.04354442,
         0.03784927, -0.00735014, -0.04301777,  0.04044653,  0.03388585,
         0.02398138],
       [ 0.04613856,  0.01403167,  0.02923531, -0.02850889,  0.03368899,
         0.0168766 ,  0.01934865, -0.04111039,  0.01275269,  0.04354442,
         0.03784927, -0.00735014, -0.04301777,  0.04044653,  0.03388585,
         0.02398138],
       [ 0.04613856,  0.01403167,  0.02923531, -0.02850889,  0.03368899,
         0.0168766 ,  0.01934865, -0.04111039,  0.01275269,  0.04354442,
         0.03784927, -0.00735014, -0.04301777,  0.04044653,  0.03388585,
        

# モデルの正確性を上げるためTrainingとValidationで割る

In [10]:
# validation set 50000건 분리
x_val = X_train[:50000]   
y_val = y_train[:50000]

# validation set을 제외한 나머지
partial_x_train = X_train[50000:]  
partial_y_train = y_train[50000:]

print(partial_x_train.shape)
print(x_val.shape)

(95237, 30, 16)
(50000, 30, 16)


Trainingデータ95237列   
Validationデータ50000列

In [12]:
partial_x_train.shape

TensorShape([95237, 30, 16])

# モデル訓練

In [11]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
              
epochs=20  # 몇 epoch를 훈련하면 좋을지 결과를 보면서 바꾸어 봅시다. 

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Epoch 1/20


ValueError: in user code:

    /home/aiffel/Downloads/exit/envs/aiffel/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /home/aiffel/Downloads/exit/envs/aiffel/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/aiffel/Downloads/exit/envs/aiffel/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/aiffel/Downloads/exit/envs/aiffel/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/aiffel/Downloads/exit/envs/aiffel/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:531 train_step  **
        y_pred = self(x, training=True)
    /home/ai